Nothing new so far

In [16]:
from transformers import TimesformerConfig, TimesformerModel, AutoImageProcessor
import torch
import os
import cv2
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torchvision

In [2]:
df = pd.read_csv("../data/mirror-data.csv")
df["Action"] = df["Action"].str.rstrip()
df = df[df.Action != "Talking&Yawning"]
df["label"] = df.Action.astype('category').cat.codes

In [3]:
i = 0
dfTrain = pd.DataFrame()
dfTest = pd.DataFrame()

while i<len(df):
    if i%5==0:
        dfTest = pd.concat([dfTest, df.iloc[[i]]])
    else :
        dfTrain = pd.concat([dfTrain, df.iloc[[i]]])
    i+=1

In [4]:
IMG_SIZE = 224
MAX_SEQ_LENGTH = 8
frame_step = 1

In [17]:
def frames_from_video_file(video_path, MAX_SEQ_LENGTH, frame_step, output_size = (IMG_SIZE, IMG_SIZE)):
    result = []
  
    reader  = torchvision.io.VideoReader(str(video_path))

    for i in range(MAX_SEQ_LENGTH):
        frame = next(reader)
        print(frame)
        result.append(frame['data'])

    return result

In [24]:
class VideoDataset(torch.utils.data.Dataset):
    def __init__(self, df, root_dir, MAX_SEQ_LENGTH, frame_step):
        self.video_paths = df["video-name"].values.tolist()
        self.labels = df["label"].values.tolist()
        self.n_frames = MAX_SEQ_LENGTH
        self.root_dir = root_dir
        self.frame_step = frame_step
        
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        path = self.video_paths[idx]
        label = self.labels[idx]
        frames = frames_from_video_file(os.path.join(self.root_dir, path), self.n_frames, self.frame_step)
        frames = torch.stack(frames)
        output = {
                'video': frames,
                'target': label
                }
        return output

In [25]:
train_ds = VideoDataset(dfTrain, "../data/YawDD/YawDD dataset/Mirror/all/", MAX_SEQ_LENGTH, frame_step)
test_ds = VideoDataset(dfTest, "../data/YawDD/YawDD dataset/Mirror/all/", MAX_SEQ_LENGTH, frame_step)

In [26]:
train_loader = DataLoader(train_ds, batch_size=4)
val_loader = DataLoader(test_ds, batch_size=4)